In [60]:
import polars as pl

In [76]:
df = pl.read_ndjson('./experiments/base/tests.ndjson', ignore_errors=True)
df = df.with_columns(experiment_type=pl.col('name').str.slice(0,3))
print(df['experiment_type'].value_counts())
df = df.explode('predicted_p_values', 'true_p_values')

ComputeError: error parsing ndjson expected value at line 682 column 152

In [72]:
df.plot.scatter(
    x='predicted_p_values',
    y='true_p_values',
    alpha=0.7,
    ylim=(-0.1,1.1),
    xlim=(-0.1,1.1),
    height=400,
    width=400,
    #by='name', width=600,
    groupby=['experiment_type', 'num_clients', 'num_samples']
    )

BokehModel(combine_events=True, render_bundle={'docs_json': {'0e445aa2-1a78-4108-9411-ac52109c4447': {'version…

In [73]:
_df = df
_df = _df.group_by('name', 'experiment_type', 'num_clients', 'num_samples') \
    .agg(pl.corr('predicted_p_values', 'true_p_values')) \
    .rename({'predicted_p_values': 'p_value_correlation'}) 
    
_df.sort('num_samples').plot.line(x='num_samples',
                                  y='p_value_correlation',
                                  alpha=0.6,
                                  by='name',
                                  groupby=['experiment_type','num_clients'],
                                  ylim=(0.5,1.001)
                                  )

BokehModel(combine_events=True, render_bundle={'docs_json': {'ac0832d4-30a8-4e1c-8931-33a7b78da5b0': {'version…

In [74]:
alpha = 0.05

_df = df
_df = _df.with_columns(
    tp=(pl.col('predicted_p_values') < alpha) & (pl.col('true_p_values') < alpha),
    tn=(pl.col('predicted_p_values') > alpha) & (pl.col('true_p_values') > alpha),
    fp=(pl.col('predicted_p_values') < alpha) & (pl.col('true_p_values') > alpha),
    fn=(pl.col('predicted_p_values') > alpha) & (pl.col('true_p_values') < alpha),
)

_df = _df.group_by('name', 'experiment_type', 'num_clients', 'num_samples').agg((pl.col('tp')+pl.col('tn')).mean().alias('accuracy'))
    
_df.sort('num_samples').plot.line(x='num_samples',
                                  y='accuracy',
                                  alpha=0.6,
                                  by='name',
                                  groupby=['experiment_type','num_clients'],
                                  ylim=(0.5,1.001)
                                  )

BokehModel(combine_events=True, render_bundle={'docs_json': {'7e2725b0-8f01-48e8-b57e-65bf11c6db96': {'version…